# Creating Marine Environment

In [1]:
import gymnasium as gym
from gymnasium import spaces
from gymnasium.envs.registration import register, registry
import numpy as np
import pygame
import time

import math
import random
import matplotlib
import matplotlib.pyplot as plt

from collections import namedtuple, deque
from itertools import count

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [2]:
if 'MarineEnv-v0' not in registry:
    register(
        id='MarineEnv-v0',
        entry_point='marine_env:MarineEnv',  # String reference to the class
    )

# Test the environment
env = gym.make('MarineEnv-v0')

In [3]:
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [4]:
Transition = namedtuple(typename='Transition', field_names=('state', 'action', 'next_state', 'reward'))

In [5]:
class ReplayMemory:
    """
    A replay buffer that stores transitions encountered by the agent.
    It enables experience replay by sampling random batches of transitions
    to break temporal correlation and improve learning stability.

    Attributes:
        capacity (int): The maximum number of transitions to store in memory.
        memory (deque): A double-ended queue to store transitions with fixed size.
    """

    def __init__(self, capacity: int):
        """
        Initializes the ReplayMemory with a fixed capacity.

        Args:
            capacity (int): The maximum number of transitions the buffer can hold.
        """
        self.memory = deque([], maxlen=capacity)

    def push(self, *args):
        """
        Add a transition to the memory buffer.

        Args:
            *args: The elements of a transition (state, action, reward, next_state, done),
                   which will be wrapped into a Transition namedtuple.
        """
        state, action, next_state, reward = args
        state = state.to(device)
        action = action.to(device)
        if next_state is not None:
            next_state = next_state.to(device)
        reward = reward.to(device)

        self.memory.append(Transition(state, action, next_state, reward))

    def sample(self, batch_size: int):
        """
        Randomly sample a batch of transitions from the memory buffer.

        Args:
            batch_size (int): The number of transitions to sample.

        Returns:
            List[Transition]: A list of randomly sampled transitions.
        """
        return random.sample(self.memory, batch_size)

    def __len__(self) -> int:
        """
        Return the current size of the memory buffer.

        Returns:
            int: The number of transitions currently stored in memory.
        """
        return len(self.memory)

In [6]:
class DQN(nn.Module):
    """
    A Deep Q-Network (DQN) implemented as a Multilayer Perceptron (MLP).

    The network approximates the Q-value function, which predicts the cumulative 
    reward for each possible action in a given state.

    Attributes:
        layer1 (nn.Linear): First fully connected layer (input to 128 neurons).
        layer2 (nn.Linear): Second fully connected layer (128 to 128 neurons).
        layer3 (nn.Linear): Output layer (128 to n_actions neurons).
    """

    def __init__(self, n_observations: int, n_actions: int):
        """
        Initializes the DQN network with three fully connected layers.

        Args:
            n_observations (int): The size of the input (state of the environment).
            n_actions (int): The number of possible actions in the environment.
        """
        super(DQN, self).__init__()
        self.layer1 = nn.Linear(n_observations, 128)  # Input to hidden layer 1
        self.layer2 = nn.Linear(128, 128)  # Hidden layer 1 to hidden layer 2
        self.layer3 = nn.Linear(128, n_actions)  # Hidden layer 2 to output

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Performs a forward pass through the network.

        Args:
            x (torch.Tensor): The input tensor representing the state of the environment.

        Returns:
            torch.Tensor: The Q-values for each possible action.
        """
        x = F.relu(self.layer1(x))  # Apply ReLU to the first layer
        x = F.relu(self.layer2(x))  # Apply ReLU to the second layer
        return self.layer3(x)  # Output layer (no activation applied)


In [7]:
# Size of the batch sampled from the replay buffer
BATCH_SIZE = 128
# Explanation:
# The number of experiences (state-action-reward transitions) sampled from the replay buffer
# at each training step. Larger batch sizes improve stability but increase computational cost.

# Discount factor for future rewards
GAMMA = 0.99
# Explanation:
# The discount factor determines how much importance is given to future rewards.
# A value close to 1 means future rewards are highly valued, while a value closer to 0 means
# the agent focuses on immediate rewards.

# Initial value for epsilon in the epsilon-greedy policy
EPS_START = 0.9
# Explanation:
# Epsilon is the probability of choosing a random action (exploration).
# This is the starting value, meaning the agent initially explores 90% of the time.

# Final value for epsilon in the epsilon-greedy policy
EPS_END = 0.05
# Explanation:
# The minimum value epsilon can decay to. At this stage, the agent mostly exploits
# the learned policy but still explores 5% of the time to avoid getting stuck in local optima.

# Decay rate for epsilon in the epsilon-greedy policy
EPS_DECAY = 1000
# Explanation:
# The rate at which epsilon decays over time. A higher value makes epsilon decay slower,
# allowing the agent to explore for longer. The effective epsilon at time step `t` can
# be calculated as:
# epsilon(t) = EPS_END + (EPS_START - EPS_END) * exp(-t / EPS_DECAY)

# Target network soft update rate
TAU = 0.005
# Explanation:
# Determines how much the target network is updated towards the online (policy) network
# at each training step. A lower TAU makes the target network update more slowly,
# stabilizing the learning process. The update rule is:
# target_param = TAU * policy_param + (1 - TAU) * target_param

# Learning rate for the optimizer
LR = 1e-04
# Explanation:
# The step size for gradient updates during backpropagation.
# A smaller learning rate ensures slow and stable learning but may require more steps to converge.
# Too large a value may lead to instability in training.

In [8]:
# Number of actions in the environment
n_actions = env.action_space.n

# Number of state observations
state, info = env.reset()
n_observations = len(state)

# Initialize the policy and target networks
policy_net = DQN(n_observations=n_observations, n_actions=n_actions).to(device)
target_net = DQN(n_observations=n_observations, n_actions=n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())  # Copy weights from policy_net to target_net

# Optimizer for training the policy network
optimizer = optim.AdamW(policy_net.parameters(), lr=LR)

# Replay memory for experience replay
memory = ReplayMemory(10000)

# Step counter for epsilon decay
steps_done = 0


In [9]:
def select_action(state: torch.Tensor) -> torch.Tensor:
    """
    Selects an action using an epsilon-greedy policy.

    Args:
        state (torch.Tensor): The current state of the environment.

    Returns:
        torch.Tensor: The selected action as a 1x1 tensor (contains the action index).
    """
    global steps_done
    sample = random.random()  # Random number to determine explore vs exploit
    eps_threshold = EPS_END + (EPS_START - EPS_END) * math.exp(-1.0 * steps_done / EPS_DECAY)
    steps_done += 1

    if sample > eps_threshold:  # Exploit: Use policy network to select the best action
        with torch.no_grad():
            return policy_net(state).max(1)[1].view(1, 1).to(device)
    else:  # Explore: Select a random action
        return torch.tensor([[env.action_space.sample()]], dtype=torch.long, device=device)

In [10]:
episode_rewards = []  # To store cumulative rewards per episode


def plot_rewards(show_result=False):
    plt.figure(1)
    rewards_t = torch.tensor(episode_rewards, dtype=torch.float)

    if show_result:
        plt.title('Result')
    else:
        plt.clf()
        plt.title('Training')

    plt.xlabel('Episode')
    plt.ylabel('Cumulative Reward')
    plt.plot(rewards_t.numpy(), label='Episode Reward')

    # Plot moving average of the last 100 episodes
    if len(rewards_t) >= 100:
        means = rewards_t.unfold(0, 100, 1).mean(1)
        means = torch.cat((torch.zeros(99), means))
        plt.plot(means.numpy(), label='100-Episode Average')

    plt.pause(0.001)
    if is_ipython:
        if not show_result:
            display.display(plt.gcf())
            display.clear_output(wait=True)
        else:
            display.display(plt.gcf())


In [11]:
def optimize_model():
    """
    Perform one step of optimization for the policy network using the replay memory.

    The function:
        - Samples a batch of transitions from the replay memory.
        - Computes the expected Q-values for the current state-action pairs.
        - Computes the loss between the expected Q-values and the predicted Q-values.
        - Performs a backward pass to update the policy network's parameters.

    Returns:
        None
    """
    if len(memory) < BATCH_SIZE:
        return  # Exit if there are not enough samples in memory to form a batch

    # Sample a batch of transitions from the replay memory
    transitions = memory.sample(BATCH_SIZE)
    batch = Transition(*zip(*transitions))  # Convert batch-array of transitions to Transition of batch-arrays

    # Compute a mask of non-final states and concatenate the batch elements
    non_final_mask = torch.tensor(
        tuple(map(lambda s: s is not None, batch.next_state)), device=device, dtype=torch.bool
    )
    non_final_next_states = torch.cat([s for s in batch.next_state if s is not None])
    state_batch = torch.cat(batch.state)
    action_batch = torch.cat(batch.action)
    reward_batch = torch.cat(batch.reward)

    # Compute Q(s_t, a) - the model's Q-values for the current states and actions
    state_action_values = policy_net(state_batch).gather(1, action_batch)

    # Initialize tensor for the next state values
    next_state_values = torch.zeros(BATCH_SIZE, device=device)

    # Compute V(s_{t+1}) for all next states
    with torch.no_grad():
        next_state_values[non_final_mask] = target_net(non_final_next_states).max(1)[0]

    # Compute expected Q-values
    expected_state_action_values = reward_batch + (GAMMA * next_state_values)

    # Compute Huber loss (Smooth L1 Loss)
    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    # Optimize the model
    optimizer.zero_grad()  # Clear the gradients from the previous step
    loss.backward()  # Backpropagate the loss

    # Clip gradients to avoid exploding gradients
    torch.nn.utils.clip_grad_value_(policy_net.parameters(), clip_value=1.0)

    # Perform a single optimization step
    optimizer.step()


In [12]:
def soft_update_target_network(policy_net, target_net, tau=0.01):
    """
    Perform a soft update of the target network parameters.

    Args:
        policy_net (nn.Module): The policy network whose parameters are used for the update.
        target_net (nn.Module): The target network to be updated.
        tau (float): The soft update coefficient. Values closer to 1.0 mean faster updates.
    """
    for target_param, policy_param in zip(target_net.parameters(), policy_net.parameters()):
        target_param.data.copy_(tau * policy_param.data + (1.0 - tau) * target_param.data)

In [ ]:
if torch.cuda.is_available():
    num_episodes = 600
else:
    num_episodes = 50

training = True

if training:
    env.unwrapped.time_scale = 10
    env.unwrapped.training = True

for i_episode in range(num_episodes):
    state, info = env.reset()
    state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)
    rewards = 0
    
    for t in count():
        action = select_action(state)
        observation, reward, terminated, truncated, _ = env.step(action.item())
        reward = torch.tensor([reward], device=device)
        rewards += reward
        done = terminated or truncated

        if terminated:
            next_state = None
        else:
            next_state = torch.tensor(observation, dtype=torch.float32, device=device).unsqueeze(0)

        # store the transition in memory
        memory.push(state, action, next_state, reward)

        # move to next state
        state = next_state

        # Update the model every N steps
        # TRAIN_EVERY = 4
        # if steps_done % TRAIN_EVERY == 0:
        optimize_model()

        # Soft update the target network
        soft_update_target_network(policy_net, target_net, tau=TAU)

        if done:
            episode_rewards.append(rewards)
            plot_rewards()
            break

print('Completed')
plot_rewards(show_result=True)
plt.ioff()
plt.show()

In [ ]:
# torch.save(policy_net.state_dict(), 'dqn_policy_1.pth')

In [13]:
policy_net.load_state_dict(torch.load("dqn_policy_1.pth"))

/tmp/ipykernel_61/1047014131.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  policy_net.load_state_dict(torch.load("dqn_policy_1.pth"))


<All keys matched successfully>

In [14]:
# Environment setup
env = gym.make('MarineEnv-v0')

# Reset the environment
state, info = env.reset()
state = torch.tensor(state, dtype=torch.float32, device=device).unsqueeze(0)

for t in range(10000):  # Maximum steps for visualization
    with torch.no_grad():
        # Use the trained policy network to select the best action
        action = policy_net(state).max(1)[1].view(1, 1).item()
        # action = env.action_space.sample()

    # Take the action in the environment
    next_state, reward, terminated, truncated, info = env.step(action)

    # Render the environment
    env.render()

    # Prepare the next state
    if not terminated:
        state = torch.tensor(next_state, dtype=torch.float32, device=device).unsqueeze(0)
    else:
        break
print(f"Simulated Time: {info['total_sim_time']} hours")
env.close()

AttributeError: 'NoneType' object has no attribute 'fill'

In [15]:
env.close()

In [ ]:
state

In [ ]:
policy_net(state)

In [ ]:
env.unwrapped.target_ship.cpa

In [ ]:
!python --version

In [ ]:
!pip install stable-baselines3[extra]

In [7]:
from stable_baselines3 import DQN
from stable_baselines3.common.evaluation import evaluate_policy

2025-01-14 21:33:06.475465: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-14 21:33:06.639978: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [8]:
sb_env = gym.make('MarineEnv-v0')

training = True

if training:
    sb_env.unwrapped.time_scale = 10
    sb_env.unwrapped.training = True



In [9]:
model = DQN(
    "MlpPolicy",
    sb_env,
    verbose=1,
    exploration_final_eps=0.1,
    target_update_interval=250,
)

Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


In [10]:
# Separate env for evaluation
eval_env = gym.make("MarineEnv-v0")
eval_env.unwrapped.time_scale *= 10
eval_env.unwrapped.training = True
# Random Agent, before training
mean_reward, std_reward = evaluate_policy(
    model,
    eval_env,
    n_eval_episodes=10,
    deterministic=True,
)

print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

/usr/local/lib/python3.8/dist-packages/stable_baselines3/common/evaluation.py:67: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


mean_reward=9423.80 +/- 17082.779345248404


In [11]:
model.learn(total_timesteps=int(2e5), progress_bar=True)

Output()

-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 2.7e+04   |
|    ep_rew_mean      | -2.52e+03 |
|    exploration_rate | 0.1       |
| time/               |           |
|    episodes         | 4         |
|    fps              | 544       |
|    time_elapsed     | 198       |
|    total_timesteps  | 107941    |
| train/              |           |
|    learning_rate    | 0.0001    |
|    loss             | 4.71      |
|    n_updates        | 26960     |
-----------------------------------
----------------------------------
| rollout/            |          |
|    ep_len_mean      | 1.7e+04  |
|    ep_rew_mean      | 8.25e+03 |
|    exploration_rate | 0.1      |
| time/               |          |
|    episodes         | 8        |
|    fps              | 535      |
|    time_elapsed     | 254      |
|    total_timesteps  | 135997   |
| train/              |          |
|    learning_rate    | 0.0001   |
|    loss             | 1.31     |
|    

In [12]:
mean_reward, std_reward = evaluate_policy(model, model.get_env(), n_eval_episodes=10)

KeyboardInterrupt: 

In [20]:
vec_env = model.get_env()
vec_env.unwrapped.time_scale = 1
obs = vec_env.reset()
for i in range(1000):
    action, _states = model.predict(obs, deterministic=True)
    obs, rewards, dones, info = vec_env.step(action)
    vec_env.render("human")

In [21]:
obs

array([[93.19180723,  7.95600933,  2.43173144, 13.53242248]])

In [22]:
model.predict(obs, deterministic=True)

(array([0]), None)

In [23]:
obs, rewards, dones, info = vec_env.step(action)

In [24]:
rewards

array([2.272057], dtype=float32)